In [1]:
import os
import sys
import time
from pathlib import Path
import requests

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib
import zipfile as zf

from service.modelss.allmodels import ApproximateNearestNeighbors

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


# LOAD DATA 

In [4]:
DATA_PATH = Path("../data/raw/kion_train/")
items = pd.read_csv(DATA_PATH / 'items.csv')

In [5]:
filename = '../models/model_LightFM.clf'
model = joblib.load(filename)
filename_ds = '../data/processed/data_DATASET.sav'
dataset = joblib.load(filename_ds)

# Nearest Neighbors

In [56]:
nearestn = ApproximateNearestNeighbors(model=model,
                                       dataset=dataset, 
                                       M=64, 
                                       num_threads=4, 
                                       efC=64, 
                                       efS=128)

In [57]:
start = time.time()
nearestn.fit(k_reco=10)
end = time.time()
print('Indexing time = %f' % (end-start))

Indexing time = 38.745573


In [58]:
nearestn.predict(user_id=0)

array([ 9728,  7829, 13865,  7571,  7626,  3734,  5693, 10440, 10942,
        3182])

In [59]:
nearestn.predict(user_id=100000)

array([15297, 10440,  4151, 13865,  3734,  4880,  2657,  9996, 12192,
        6809])

In [45]:
filename = '../models/model_LightFM_ANN.clf'
joblib.dump(model, filename)

['model_LightFM_ANN.clf']

# Проверка рекомендаций аватаров

In [46]:
# Проверим рекомендации для наших аватаров и случайного юзера

id_avatars = [100000, 200000, 400000, 13]

#predict_model
recos = model.recommend(
    users=id_avatars,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [60]:
user_1 = recos.query("user_id == 100000")['item_id'].values
user_2 = recos.query("user_id == 200000")['item_id'].values
user_3 = recos.query("user_id == 400000")['item_id'].values

user_1_1 = nearestn.predict(user_id=100000)
user_1_2 = nearestn.predict(user_id=200000)
user_1_3 = nearestn.predict(user_id=400000)

### 1й пользователь, который смотрит только новые фильмы, с 2015 года, модель LightFM порекомендовала довольно новые

In [48]:
items.query("item_id in @user_1")

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
9253,1844,film,Аферистка,I Care A Lot,2020.0,"триллеры, комедии",Великобритания,NaN,18.0,NaN,Джей Блейксон,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...",Марла руководствуется принципом «побеждает сил...,"опекун, аферист, чёрная комедия, мошенник, 202..."
9558,7793,film,Радиовспышка,Radioflash,2019.0,"боевики, драмы, фантастика, триллеры",США,NaN,16.0,NaN,Бен Макферсон,"Брайтон Шарбино, Доминик Монахэн, Уилл Пэттон,...","Риз с легкостью проходит виртуальные квесты, н...","2019, соединенные штаты, радиовспышка"
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
14383,4457,film,2067: Петля времени,2067,2020.0,"драмы, фантастика",Австралия,NaN,16.0,NaN,Сет Ларни,"Аарон Гленнэйн, Коди Смит-МакФи, Райан Квантен...",2067 год. Люди окончательно загубили экологию ...,"изменение климата, путешествие во времени, выр..."
14958,7102,film,Дочь волка,Daughter of the Wolf,2019.0,"боевики, триллеры",Канада,NaN,18.0,NaN,Дэвид Хэкл,"Джина Карано, Брендан Фер, Ричард Дрейфусс, Ст...",Клэр возвращается домой из поездки по Ближнему...,"2019, канада, дочь, волка"
15234,142,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма..."


### модель LightFM+ANN порекомендовала тоже новые фильмы, но с меньши разнообразием в годах 

In [49]:
items.query("item_id in @user_1_1")

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
202,4880,series,Афера,Afera,2021.0,комедии,Россия,NaN,18.0,NaN,Михаил Старчак,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Смотри:- как кино- как сериалКарантин окончен,...","Афера, Аферисты, Карантин, Пандемия, Карантин ..."
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
3875,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
7279,9996,series,Немцы,Nemtsy,2021.0,драмы,Россия,NaN,16.0,NaN,Стас Иванов,"Евгений Коряковский, Анна Завтур, Антон Василь...",На журналиста-оппозиционера Антона Эбергарда с...,"немцы, немец, про немцев, по мотивам романа, п..."
10604,12192,series,Фемида видит,Femida vidit,2019.0,"драмы, детективы, комедии",Россия,NaN,16.0,NaN,"Никита Грамматиков, Ирина Бас","Анна Котова-Дерябина, Александр Давыдов, Алекс...",Лёгкий комедийный детектив от автора «Подслуша...,"фемида видит, фемида, феми, фем, фемид, фемида..."
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
13615,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ..."


### 2й пользователь, который смотрит только фэнтези

In [50]:
df = items.query("item_id in @user_2")
df

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
1274,5411,film,Монстры на каникулах 3: Море зовёт,Hotel Transylvania 3: Summer Vacation,2018.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Генндий Тартаковский,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...",На этот раз Дракула вместе со своими друзьями ...,"третья часть, круизное судно, персонаж Дракула..."
1554,16270,film,Тайна Коко,Coco,2017.0,"мультфильм, фэнтези, приключения",США,NaN,12.0,NaN,"Ли Анкрич, Эдриан Молина","Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...",Мексиканский юноша Мигель живёт в семье сапожн...,"Мексика, гитара, музыкант, скелет, музыка, заг..."
1849,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня..."
3951,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ..."
7485,13955,film,Душа,Soul,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...",США,NaN,6.0,NaN,"Пит Доктер, Кемп Пауэрс","Джейми Фокс, Тина Фей, Грэм Нортон, Рэйчел Хау...",Режиссёр «Вверх» и «Головоломки» вновь сделал ...,"нью-йорк, джаз, самопознание, джазовый певец и..."
8275,2954,film,Миньоны,Minions,2015.0,"фантастика, мультфильм, приключения, комедии",США,NaN,6.0,NaN,"Кайл Балда, Пьер Коффан","Сандра Буллок, Джон Хэмм, Майкл Китон, Эллисон...",Миньоны живут на планете гораздо дольше нас. У...,"помощник, сцена после титров, сцена во время т..."
9085,1105,film,Рапунцель: Запутанная история,Tangled,2010.0,"мультфильм, приключения, мюзиклы, мелодрамы, ф...",США,NaN,6.0,NaN,"Нэйтан Грено, Байрон Ховард","Мэнди Мур, Закари Ливай, Донна Мерфи, Рон Перл...",Обаятельный разбойник Флинн путешествует по жи...,"Заложник, магия, лошадь, сказка, мюзикл, блонд..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел..."
12035,15266,film,Корпорация монстров,Monsters University,2001.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,"Пит Доктер, Дэвид Силверман, Ли Анкрич","Джон Гудман, Билли Кристал, Мэри Гиббс, Стив Б...","Склизкий гад в сливном бачке, мохнатый зверь, ...","чудовище, обман, похищение, младенец, злодей, ..."
13577,7310,film,Гадкий я 2,Despicable Me 2,2013.0,"мультфильм, приключения, фантастика, фэнтези, ...","США, Франция, Япония",NaN,0.0,NaN,"Пьер Коффан, Крис Рено","Стив Карелл, Кристен Уиг, Бенджамин Брэтт, Мир...","В то время как Грю, бывший суперзлодей, приспо...","отношения родитель-ребенок, секретный агент, п..."


### и как видно LightFM немного отходит от заданного жанра

In [51]:
df.genres.apply(lambda x: True if 'фэнтези' in x else False)

1274      True
1554      True
1849      True
3951     False
7485      True
8275     False
9085      True
9698     False
12035     True
13577     True
Name: genres, dtype: bool

### a LightFM+ANN совсем плохо справляется

In [52]:
df = items.query("item_id in @user_1_2")
df.genres.apply(lambda x: True if 'фэнтези' in x else False)

202      False
767      False
1849      True
1882     False
3951     False
6346     False
6501     False
6689     False
10087     True
12050    False
Name: genres, dtype: bool

### 3й пользователь смотрит фильмы США

In [53]:
df = items.query("item_id in @user_3")
df

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
1849,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6411,14317,film,Веном,Venom,2018.0,"популярное, фантастика, триллеры, боевики, ужасы",США,NaN,16.0,NaN,Рубен Фляйшер,"Том Харди, Вуди Харрельсон, Уэйд Уильямс, Мише...",Что если в один прекрасный день в тебя вселяет...,"Сан-Франциско, Калифорния, космический корабль..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
9477,14741,film,Цвет из иных миров,Colour out of space,2020.0,"фантастика, ужасы","США, Малайзия, Португалия",NaN,16.0,NaN,Ричард Стэнли,"Николас Кейдж, Джоэли Ричардсон, Мадлен Артур,...",Экранизация рассказа Говарда Лавкрафта про упа...,"мутация, хижина, сарай, отшельник, ферма, мете..."
9558,7793,film,Радиовспышка,Radioflash,2019.0,"боевики, драмы, фантастика, триллеры",США,NaN,16.0,NaN,Бен Макферсон,"Брайтон Шарбино, Доминик Монахэн, Уилл Пэттон,...","Риз с легкостью проходит виртуальные квесты, н...","2019, соединенные штаты, радиовспышка"
10087,7829,film,Поступь хаоса,Chaos Walking,2021.0,"боевики, фантастика, фэнтези, приключения","США, Канада, Люксембург",NaN,16.0,NaN,Даг Лайман,"Том Холланд, Дэйзи Ридли, Демиан Бичир, Дэвид ...",2257 год. Родина Тодда Хьюитта — колонизирован...,"по роману или книге, постапокалиптическое буду..."
10725,7626,film,Мстители: Война бесконечности,AVENGERS: INFINITY WAR,2018.0,"боевики, фантастика, приключения",США,NaN,16.0,NaN,"Энтони Руссо, Джо Руссо","Роберт Дауни мл., Крис Хемсворт, Марк Руффало,...",Пока Мстители и их союзники продолжают защищат...,"магия, жертва, супергерой, основанный на комик..."
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"


###  LightFM порекомендовала фильмы США почти с таким расхождением, что и по жарнам

In [54]:
df.countries.apply(lambda x: True if 'США' in x else False)

1849     False
1882     False
6346      True
6411      True
6501     False
9477      True
9558      True
10087     True
10725     True
12050    False
Name: countries, dtype: bool

###  LightFM+ANN совсем не попала

In [61]:
df = items.query("item_id in @user_1_3")
df.countries.apply(lambda x: True if 'США' in x else False)

202      False
767      False
1882     False
6346      True
6501     False
6689     False
9253     False
12050    False
13615    False
15234    False
Name: countries, dtype: bool